In [1]:
pip install datasets

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from datasets import load_dataset
import pandas as pd
import numpy as  np
import re

# Load the dataset and select the first 100,000 rows, then take a subset of 1,000
ds = load_dataset("MohammadOthman/mo-customer-support-tweets-945k")
data = ds['train'].select(range(200000)).to_pandas()
# data_subset = data.head(1000)  # Subset for demonstration
data_subset = data

####################################
# Drop rows with NaN values in 'input' or 'output'
data_subset.dropna(subset=['input', 'output'], inplace=True)

# Drop rows with empty strings or only one word in either 'input' or 'output'
def is_valid(text):
    return isinstance(text, str) and len(text.strip().split()) > 1

data_subset = data_subset[data_subset['input'].apply(is_valid)]
data_subset = data_subset[data_subset['output'].apply(is_valid)]

# Reset index after filtering
data_subset.reset_index(drop=True, inplace=True)

print("Cleaned subset shape:", data_subset.shape)
############################################################


# Preprocess text: lowercase and remove special characters
def preprocess_text(text):
    if not isinstance(text, str):  # Handle non-string inputs
        return ""
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text.strip()

# Apply preprocessing to input and output columns
data_subset['input'] = data_subset['input'].apply(preprocess_text)
data_subset['output'] = data_subset['output'].apply(preprocess_text)

print("Subset shape:", data_subset.shape)

/Users/ritikojha/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Cleaned subset shape: (193274, 2)
Subset shape: (193274, 2)


In [2]:
# data_subset[['output', 'input']].to_csv('main_data.csv', index = False)

In [3]:
data_subset.head()

,output,input
0,i would love the chance to review the account ...,is the worst customer service
1,h there we would definitely like to work with ...,yall lie about your great connection 5 bars lt...
2,we understand your concerns and we would like ...,since i signed up with yousince day 1
3,this is saddening to hear please shoot us a dm...,you gonna magically change your connectivity f...
4,please send me a private message so that i can...,whenever i contact customer support they tell ...


In [4]:

# output-- do precessing try to search some metho dto see if the language is other than english, and then i have seen somewhere wghere the words were spell wrong(try some simple method to correct the word)
# embeddings--don't do bert, try faster static embeddings from hugging face(or any faster embedding), or try tfidf later on.
# clustering on embedding-- kmeans(kmeans will be bad because of higher dimension.., for another option, try reducing dimension by pca, pls, and then on those reduced dimension , do kmeans) try other clustering methods like tsne, umap, try other fast clustering which is good for higher dimension

# in future--, for each cluster, we will try to implement something like different kind chatbiot we can develop


In [5]:
data_subset['conversation'] = data_subset['input'] + " " +data_subset['output']

In [6]:
import pandas as pd
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException
from pandarallel import pandarallel

# Ensures consistent language detection
DetectorFactory.seed = 0

# Initialize pandarallel
pandarallel.initialize(progress_bar=True)  # Set to False to hide progress bar

# Sample large DataFrame (replace this with your actual one)
# df = pd.read_csv("your_large_file.csv")
# Let's say the relevant column is 'text'
# For demo:
# df = pd.DataFrame({'text': [...large text list...]})

# Function to check if text is English
def is_english(text):
    try:
        return detect(text) == 'en'
    except LangDetectException:
        return False

# Use parallel apply
# df['is_english'] = df['text'].parallel_apply(is_english)

# # Filter for English rows only
# df_english = df[df['is_english']].drop(columns='is_english').reset_index(drop=True)

# # Done!
# print(df_english.head())


INFO: Pandarallel will run on 11 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [7]:
data_subset['is_english'] = data_subset['conversation'].parallel_apply(is_english)

# Filter for English rows only
data_subset = data_subset[data_subset['is_english']].drop(columns='is_english').reset_index(drop=True)

# Done!


In [8]:
data_subset.shape

(185457, 3)

In [ ]:
# from transformers import BertTokenizer, BertModel
# import torch
# import numpy as np

# # Load BERT tokenizer and model
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertModel.from_pretrained('bert-base-uncased')

# # Function to generate BERT embeddings
# def get_bert_embedding(text):
#     if not text:  # Handle empty strings
#         return np.zeros(768)  # Return zero vector for empty text
#     inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
#     with torch.no_grad():
#         outputs = model(**inputs)
#     # Use the [CLS] token embedding
#     return outputs.last_hidden_state[:, 0, :].squeeze().numpy()

# # Generate embeddings for agent responses
# embeddings_list = [get_bert_embedding(text) for text in data_subset['conversation']]
# embeddings_matrix = np.array(embeddings_list)

# print("Embeddings matrix shape:", embeddings_matrix.shape)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch, numpy as np
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

# 1) Setup
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model     = DistilBertModel.from_pretrained('distilbert-base-uncased')
device    = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device).eval()

# 2) Dataset returns raw text
class ConversationDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        # Return the raw string (empty→"")
        return self.texts[idx] or ""

# 3) collate_fn: batch→tokenized, padded tensors
def collate_fn(batch_texts):
    # batch_texts: list[str]
    encoded = tokenizer(
        batch_texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    )
    # move to device
    return {k: v.to(device) for k, v in encoded.items()}

# 4) Embedding function
@torch.no_grad()
def get_bert_embeddings_batch(batch):
    outputs = model(**batch)
    # take [CLS] token (first token) out, move to CPU numpy
    return outputs.last_hidden_state[:, 0, :].cpu().numpy()

# 5) DataLoader with custom collate_fn
batch_size = 64
dataset    = ConversationDataset(data_subset['conversation'].tolist())
dataloader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=collate_fn,  # ← ensures uniform padding
    num_workers=0           # safe for notebooks
)

# 6) Generate embeddings
emb_list = []
for batch in tqdm(dataloader, desc="Embedding batches"):
    emb = get_bert_embeddings_batch(batch)
    emb_list.append(emb)

embeddings_matrix = np.vstack(emb_list)
print("Embeddings matrix shape:", embeddings_matrix.shape)


Embedding batches:  85%|████████▍ | 2452/2898 [22:07<04:50,  1.54it/s]

: 

In [18]:
# # distilbert
# # Import required libraries
# from transformers import DistilBertTokenizer, DistilBertModel
# import torch
# import numpy as np

# # Load DistilBERT tokenizer and model
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
# model = DistilBertModel.from_pretrained('distilbert-base-uncased')

# # Move model to GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)
# model.eval()

# # Function to generate DistilBERT embeddings using GPU
# def get_distilbert_embedding(text):
#     if not text:
#         return np.zeros(768)  # Handle empty or null strings

#     # Tokenize and move tensors to device
#     inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

#     with torch.no_grad():
#         outputs = model(**inputs)

#     # Get the [CLS] token embedding, move back to CPU before converting to NumPy
#     cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
#     return cls_embedding

# # Generate embeddings for agent responses with progress bar
# from tqdm import tqdm

# embeddings_list = [get_distilbert_embedding(text) for text in tqdm(data_subset['conversation'])]

# # Convert embeddings list to a NumPy array
# embeddings_matrix = np.array(embeddings_list)

# # Print the shape of the embeddings matrix
# print("Embeddings matrix shape:", embeddings_matrix.shape)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

: 

In [17]:
# bert network, big transformer network--

# from transformers import BertTokenizer, BertModel
# import torch
# import numpy as np

# # Load BERT tokenizer and model
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertModel.from_pretrained('bert-base-uncased')

# # Move model to GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)
# model.eval()

# # Function to generate BERT embeddings using GPU
# def get_bert_embedding(text):
#     if not text:
#         return np.zeros(768)  # Handle empty or null strings

#     # Tokenize and move tensors to device
#     inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

#     with torch.no_grad():
#         outputs = model(**inputs)

#     # Get the [CLS] token embedding, move back to CPU before converting to NumPy
#     cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
#     return cls_embedding

# # Generate embeddings for agent responses
# from tqdm import tqdm

# embeddings_list = [get_bert_embedding(text) for text in tqdm(data_subset['conversation'])]

# # embeddings_list = [get_bert_embedding(text) for text in data_subset['conversation']]
# embeddings_matrix = np.array(embeddings_list)

# print("Embeddings matrix shape:", embeddings_matrix.shape)


  0%|          | 0/466773 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# saving and loading embeddings
import numpy as np
# Save the embeddings matrix to a file
np.save("/content/drive/MyDrive/project_fdm/bert_embeddings_cluster_collab.npy", embeddings_matrix)
print("Embeddings saved to 'bert_embeddings.npy'")


In [3]:
# loading embeddings
# Load the saved embeddings
loaded_embeddings = np.load("/content/drive/MyDrive/project_fdm/bert_embeddings_cluster_collab.npy")
print("Loaded embeddings shape:", loaded_embeddings.shape)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/project_fdm/bert_embeddings_cluster_collab.npy'

Best number of clusters (based on silhouette): 2


In [ ]:
import umap.umap_ as umap

# Reduce to 50 dimensions with UMAP
umap_reducer = umap.UMAP(n_components=50, random_state=42, n_neighbors=15, min_dist=0.1)
reduced_embeddings_umap = umap_reducer.fit_transform(embeddings_matrix)

print("UMAP reduced shape:", reduced_embeddings_umap.shape)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


UMAP reduced shape: (96391, 50)


In [ ]:
from sklearn.decomposition import TruncatedSVD

# Reduce to 50 dimensions with SVD
svd = TruncatedSVD(n_components=50, random_state=42)
reduced_embeddings_svd = svd.fit_transform(embeddings_matrix)

print("SVD reduced shape:", reduced_embeddings_svd.shape)

SVD reduced shape: (96391, 50)


In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
reduced_embeddings_tsne = tsne.fit_transform(embeddings_matrix)


In [ ]:
from sklearn.cluster import MiniBatchKMeans

# On UMAP-reduced data
minibatch_kmeans_umap = MiniBatchKMeans(n_clusters=4, random_state=42, batch_size=100)
clusters_minibatch_umap = minibatch_kmeans_umap.fit_predict(reduced_embeddings_umap)

# On SVD-reduced data
minibatch_kmeans_svd = MiniBatchKMeans(n_clusters=4, random_state=42, batch_size=100)
clusters_minibatch_svd = minibatch_kmeans_svd.fit_predict(reduced_embeddings_svd)

minibatch_kmeans_tsne = MiniBatchKMeans(n_clusters=4, random_state=42, batch_size=100)
clusters_minibatch_tsne = minibatch_kmeans_tsne.fit_predict(reduced_embeddings_tsne)

In [ ]:
from sklearn.cluster import KMeans

# On UMAP-reduced data
kmeans_umap = KMeans(n_clusters=4, random_state=42)
clusters_kmeans_umap = kmeans_umap.fit_predict(reduced_embeddings_umap)

# On SVD-reduced data
# kmeans_svd = KMeans(n_clusters=4, random_state=42)
# clusters_kmeans_svd = kmeans_svd.fit_predict(reduced_embeddings_svd)

# # On tsne-reduced data
# kmeans_tsne = KMeans(n_clusters=4, random_state=42)
# clusters_kmeans_tsne = kmeans_tsne.fit_predict(reduced_embeddings_tsne)



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Assuming these exist:
# reduced_embeddings_umap: 2D numpy array from UMAP
# clusters_kmeans_umap: predicted cluster labels from KMeans

# Number of clusters
num_clusters = len(np.unique(clusters_kmeans_umap))

# Set Seaborn style
sns.set(style="white", context="notebook")

# Create the plot
plt.figure(figsize=(10, 7))
scatter = plt.scatter(
    reduced_embeddings_umap[:, 0],
    reduced_embeddings_umap[:, 1],
    c=clusters_kmeans_umap,
    cmap=plt.cm.get_cmap('tab20', num_clusters),
    s=60,
    alpha=0.8,
    linewidths=0.3,
    edgecolors='gray'
)

# Legend with cluster IDs
legend1 = plt.legend(*scatter.legend_elements(num=num_clusters), title="Cluster", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.gca().add_artist(legend1)

# Clean axis and layout
sns.despine()
plt.xlabel("UMAP Dimension 1", fontsize=12)
plt.ylabel("UMAP Dimension 2", fontsize=12)
plt.title("Enhanced KMeans Clustering on UMAP-Reduced Embeddings", fontsize=14)
plt.grid(False)
plt.tight_layout()
plt.show()


In [ ]:
# cluster ceenter
# Optional: If you have the fitted KMeans object
centers = kmeans_umap.cluster_centers_  # from the fitted KMeans on UMAP embeddings
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.7, marker='X', label='Centers')
plt.legend()



In [ ]:
from sklearn.cluster import DBSCAN

# On UMAP-reduced data
dbscan_umap = DBSCAN(eps=0.5, min_samples=5)
clusters_dbscan_umap = dbscan_umap.fit_predict(reduced_embeddings_umap)

# On SVD-reduced data
dbscan_svd = DBSCAN(eps=0.5, min_samples=5)
clusters_dbscan_svd = dbscan_svd.fit_predict(reduced_embeddings_svd)

dbscan_tsne = DBSCAN(eps=0.5, min_samples=5)
clusters_dbscan_tsne = dbscan_tsne.fit_predict(reduced_embeddings_tsne)

In [ ]:
from sklearn.metrics import silhouette_score

# Function to safely compute silhouette score
def compute_silhouette(data, labels):
    unique_labels = len(set(labels)) - (1 if -1 in labels else 0)  # Exclude noise for DBSCAN
    if unique_labels > 1:  # Need at least 2 clusters for silhouette score
        return silhouette_score(data, labels)
    return None

# Dictionary of methods and their data/labels
methods = {
    'Mini-Batch K-Means (UMAP)': (reduced_embeddings_umap, clusters_minibatch_umap),
    'Mini-Batch K-Means (SVD)': (reduced_embeddings_svd, clusters_minibatch_svd),
    'Mini-Batch K-Means (tsne)': (reduced_embeddings_tsne, clusters_minibatch_tsne),
    'K-Means (UMAP)': (reduced_embeddings_umap, clusters_kmeans_umap),
    'K-Means (SVD)': (reduced_embeddings_svd, clusters_kmeans_svd),
    'K-Means (tsne)': (reduced_embeddings_tsne, clusters_kmeans_tsne),
    'DBSCAN (UMAP)': (reduced_embeddings_umap, clusters_dbscan_umap),
    'DBSCAN (SVD)': (reduced_embeddings_svd, clusters_dbscan_svd),
    'DBSCAN (tsne)': (reduced_embeddings_tsne, clusters_dbscan_tsne),
    # 'Spectral Clustering (UMAP)': (reduced_embeddings_umap, clusters_spectral_umap),
    # 'Spectral Clustering (SVD)': (reduced_embeddings_svd, clusters_spectral_svd)
}

# Compute and print silhouette scores
for method, (data, labels) in methods.items():
    score = compute_silhouette(data, labels)
    num_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    if score is not None:
        print(f"{method}: Silhouette Score = {score:.4f}, Clusters = {num_clusters}")
    else:
        print(f"{method}: Cannot compute silhouette score (Clusters = {num_clusters})")


#Mini-Batch K-Means (UMAP): Silhouette Score = 0.3382, Clusters = 4
# Mini-Batch K-Means (SVD): Silhouette Score = 0.0919, Clusters = 4
# K-Means (UMAP): Silhouette Score = 0.486  1, Clusters = 4
# K-Means (SVD): Silhouette Score = 0.0793, Clusters = 4
# DBSCAN (UMAP): Silhouette Score = -0.3241, Clusters = 36
# DBSCAN (SVD): Silhouette Score = -0.0047, Clusters = 5

Mini-Batch K-Means (UMAP): Silhouette Score = 0.3382, Clusters = 4
Mini-Batch K-Means (SVD): Silhouette Score = 0.0919, Clusters = 4
Mini-Batch K-Means (tsne): Silhouette Score = 0.3704, Clusters = 4
K-Means (UMAP): Silhouette Score = 0.4861, Clusters = 4
K-Means (SVD): Silhouette Score = 0.0793, Clusters = 4
K-Means (tsne): Silhouette Score = 0.3865, Clusters = 4
DBSCAN (UMAP): Silhouette Score = -0.3241, Clusters = 36
DBSCAN (SVD): Silhouette Score = -0.0047, Clusters = 5
DBSCAN (tsne): Silhouette Score = 0.0729, Clusters = 4253


In [ ]:
from sklearn.cluster import KMeans

# On UMAP-reduced data
kmeans_umap = KMeans(n_clusters=4, random_state=42)
clusters_kmeans_umap = kmeans_umap.fit_predict(reduced_embeddings_umap)

# On SVD-reduced data
kmeans_svd = KMeans(n_clusters=4, random_state=42)
clusters_kmeans_svd = kmeans_svd.fit_predict(reduced_embeddings_svd)

In [ ]:
from sklearn.cluster import DBSCAN

# On UMAP-reduced data
dbscan_umap = DBSCAN(eps=0.5, min_samples=5)
clusters_dbscan_umap = dbscan_umap.fit_predict(reduced_embeddings_umap)

# On SVD-reduced data
dbscan_svd = DBSCAN(eps=0.5, min_samples=5)
clusters_dbscan_svd = dbscan_svd.fit_predict(reduced_embeddings_svd)

In [ ]:
from sklearn.cluster import SpectralClustering

# On UMAP-reduced data
spectral_umap = SpectralClustering(n_clusters=3, random_state=42, affinity='nearest_neighbors')
clusters_spectral_umap = spectral_umap.fit_predict(reduced_embeddings_umap)

# On SVD-reduced data
spectral_svd = SpectralClustering(n_clusters=3, random_state=42, affinity='nearest_neighbors')
clusters_spectral_svd = spectral_svd.fit_predict(reduced_embeddings_svd)

/Users/ritikojha/Library/Python/3.9/lib/python/site-packages/sklearn/manifold/_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/Users/ritikojha/Library/Python/3.9/lib/python/site-packages/sklearn/manifold/_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


In [ ]:
from sklearn.metrics import silhouette_score

# Function to compute silhouette score safely
def compute_silhouette(data, labels):
    unique_labels = len(set(labels))
    if unique_labels > 1 and unique_labels < len(data):  # Valid clustering
        return silhouette_score(data, labels)
    return None

# Dictionary of methods and their data/labels
methods = {
    'Mini-Batch K-Means (UMAP)': (reduced_embeddings_umap, clusters_minibatch_umap),
    'Mini-Batch K-Means (SVD)': (reduced_embeddings_svd, clusters_minibatch_svd),
    'K-Means (UMAP)': (reduced_embeddings_umap, clusters_kmeans_umap),
    'K-Means (SVD)': (reduced_embeddings_svd, clusters_kmeans_svd),
    'DBSCAN (UMAP)': (reduced_embeddings_umap, clusters_dbscan_umap),
    'DBSCAN (SVD)': (reduced_embeddings_svd, clusters_dbscan_svd),
    # 'Spectral Clustering (UMAP)': (reduced_embeddings_umap, clusters_spectral_umap),
    # 'Spectral Clustering (SVD)': (reduced_embeddings_svd, clusters_spectral_svd)
}

# Compute and print silhouette scores
for method, (data, labels) in methods.items():
    score = compute_silhouette(data, labels)
    num_clusters = len(set(labels)) - (1 if -1 in labels else 0)  # Exclude noise for DBSCAN
    if score is not None:
        print(f"{method}: Silhouette Score = {score:.4f}, Clusters = {num_clusters}")
    else:
        print(f"{method}: Cannot compute silhouette score (Clusters = {num_clusters})")

# Mini-Batch K-Means (UMAP): Silhouette Score = 0.3382, Clusters = 4
# Mini-Batch K-Means (SVD): Silhouette Score = 0.0919, Clusters = 4
# K-Means (UMAP): Silhouette Score = 0.4861, Clusters = 4
# K-Means (SVD): Silhouette Score = 0.0793, Clusters = 4
# DBSCAN (UMAP): Silhouette Score = -0.3241, Clusters = 36
# DBSCAN (SVD): Silhouette Score = -0.0047, Clusters = 5

Mini-Batch K-Means (UMAP): Silhouette Score = 0.3382, Clusters = 4
Mini-Batch K-Means (SVD): Silhouette Score = 0.0919, Clusters = 4
K-Means (UMAP): Silhouette Score = 0.4861, Clusters = 4
K-Means (SVD): Silhouette Score = 0.0793, Clusters = 4
DBSCAN (UMAP): Silhouette Score = -0.3241, Clusters = 36
DBSCAN (SVD): Silhouette Score = -0.0047, Clusters = 5


In [ ]:
# Add cluster labels to the subset
data_subset['kmeans_umap_cluster'] = clusters_kmeans_umap

# Print 2 examples from each cluster
for cluster in range(3):
    print(f"\nCluster {cluster} (K-Means UMAP):")
    cluster_data = data_subset[data_subset['kmeans_umap_cluster'] == cluster]['output'].head(2)
    for i, text in enumerate(cluster_data):
        print(f"Example {i+1}: {text}")


Cluster 0 (K-Means UMAP):
Example 1: lamentamos el inconveniente por favor siguenos en twitter para brindarte asistencia por mensajes directos
Example 2: hola ezequiel por favor siguenos en twitter para poder brindarte asistencia por mensajes directos

Cluster 1 (K-Means UMAP):
Example 1: i would love the chance to review the account and provide assistance
Example 2: h there we would definitely like to work with you on this how long have you been eeriencing this issue

Cluster 2 (K-Means UMAP):
Example 1: what information is incorrect
Example 2: no thank you


In [ ]:
# data_subset.to_csv('final_submission.csv', index = False)